In [1]:
!pip install spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 612 kB 2.0 MB/s eta 0:00:01
  Created wheel for regex: filename=regex-2018.1.10-cp36-cp36m-linux_x86_64.whl size=552157 sha256=8fcd9e619b505765b7f8951317992b3e6271ad2838085867ada9ba4db1421562
  Stored in directory: /home/work/.cache/pip/wheels/4b/72/79/772fa2486e5eafc62bd47cd83bf724c129569dd09c24d55cc0
Successfully built regex
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 39.8 MB 525 kB/s eta 0:00:01
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.0.0-py3-none-any.whl size=39841462 sha256=66048c51f08c9af319c2b2a849f0003f4007c690d383ce0987c85b093a4b0752
  Stored in directory: /tmp/pip-ephem-wheel-cache-j51nyss_/wheels/46/df/13/b060ba26a65a7cb17e5e7855de0251724fc2862d7843e8af01
Successfully built fr-core-news-sm

    Error: Couldn't link model to 'fr_core_news_sm'
    Creating a symlin

-------

In [1]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

In [2]:
spacy_en = spacy.load('en_core_web_sm')
spacy_fr = spacy.load('fr_core_news_sm')

def tokenize_en(sentence):
    return [token.text for token in spacy_en.tokenizer(sentence)]

# REVERSE SENTENCES
def tokenize_fr(sentence):
    return [token.text for token in spacy_fr.tokenizer(sentence)]

# lower를 굳이 안해도 될까? 데이터가 많기 때문 (논문 내용)
TRG = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
SRC = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [3]:
from datetime import datetime
print(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))

2021-03-24-19:11:24


In [4]:
# associate the text in the 'English' trg column with the TRG field, # and 'French' src column with SRC field
data_fields = [('trg', TRG), ('src', SRC)]
print(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))

train_data, test_data = TabularDataset.splits(path='.',
                                  train = 'train_data.csv',
                                  test = 'test_data.csv',
                                  format = 'csv',
                                  fields = data_fields,
                                  skip_header = True )

print(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))

2021-03-24-19:11:24
2021-03-24-21:02:05


In [5]:
TRG.build_vocab(train_data, min_freq=2)
SRC.build_vocab(train_data, min_freq=2)

In [6]:
print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 65290
len(TRG): 92144


In [7]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["good"])
print(TRG.vocab.stoi["world"])

0
1
2
3
43876
47036


In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 메모리 부족으로 batch_size를 너무 줄임 
BATCH_SIZE = 8

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, test_data),
    sort = False, # Tabular -> error. 
    batch_size = BATCH_SIZE,
    device = device)

In [9]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([85, 8])
인덱스 0: 2
인덱스 1: 73
인덱스 2: 85
인덱스 3: 205
인덱스 4: 5
인덱스 5: 720
인덱스 6: 14
인덱스 7: 17
인덱스 8: 64
인덱스 9: 224
인덱스 10: 5
인덱스 11: 643
인덱스 12: 39
인덱스 13: 5
인덱스 14: 8034
인덱스 15: 4
인덱스 16: 1072
인덱스 17: 321
인덱스 18: 271
인덱스 19: 101
인덱스 20: 31
인덱스 21: 4
인덱스 22: 3630
인덱스 23: 27
인덱스 24: 67
인덱스 25: 5
인덱스 26: 713
인덱스 27: 24
인덱스 28: 144
인덱스 29: 4
인덱스 30: 751
인덱스 31: 11
인덱스 32: 49
인덱스 33: 10
인덱스 34: 4
인덱스 35: 587
인덱스 36: 362
인덱스 37: 5
인덱스 38: 225
인덱스 39: 281
인덱스 40: 41
인덱스 41: 628
인덱스 42: 8
인덱스 43: 13
인덱스 44: 192
인덱스 45: 1028
인덱스 46: 485
인덱스 47: 9
인덱스 48: 13
인덱스 49: 2532
인덱스 50: 1047
인덱스 51: 14
인덱스 52: 106
인덱스 53: 1632
인덱스 54: 1028
인덱스 55: 3304
인덱스 56: 789
인덱스 57: 4
인덱스 58: 53
인덱스 59: 49
인덱스 60: 5
인덱스 61: 47
인덱스 62: 36
인덱스 63: 4
인덱스 64: 557
인덱스 65: 7
인덱스 66: 11
인덱스 67: 154
인덱스 68: 3630
인덱스 69: 27
인덱스 70: 67
인덱스 71: 8
인덱스 72: 4
인덱스 73: 49
인덱스 74: 438
인덱스 75: 9
인덱스 76: 4833
인덱스 77: 49
인덱스 78: 2575
인덱스 79: 7
인덱스 80: 4
인덱스 81: 2825
인덱스 82: 546
인덱스 83: 6
인덱스 84: 3


In [10]:
import torch.nn as nn

# 인코더(Encoder) 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환        
    def forward(self, src):
        # src: [단어 개수, 배치 크기]: 각 단어의 인덱스(index) 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 문맥 벡터(context vector) 반환
        return hidden, cell

In [11]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding) 말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)
        
        # FC 레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환     
    def forward(self, input, hidden, cell):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [레이어 개수, 배치 크기, 히든 차원]
        # cell = context: [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)
        # input: [단어 개수 = 1, 배치 크기]
        
        embedded = self.dropout(self.embedding(input))
        # embedded: [단어 개수, 배치 크기, 임베딩 차원]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

        # 단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [배치 크기, 출력 차원]
        
        # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기
        
        return outputs

In [13]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)
# ENCODER_EMBED_DIM = 256
# DECODER_EMBED_DIM = 256
# HIDDEN_DIM = 512
# N_LAYERS = 2
# ENC_DROPOUT_RATIO = 0.5
# DEC_DROPOUT_RATIO = 0.5



INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

# enc,dec dim을 줄임... 너무 안돌아가서 빨리 돌려보고 싶어서..
ENCODER_EMBED_DIM = 64
DECODER_EMBED_DIM = 64
HIDDEN_DIM = 128

N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [14]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device)
model = nn.DataParallel(model).to(device)

In [15]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(65290, 64)
    (rnn): LSTM(64, 128, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(92144, 64)
    (rnn): LSTM(64, 128, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=128, out_features=92144, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,425,200 trainable parameters


In [17]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src.to(device)
        trg = batch.trg.to(device)
        
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [18]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src.to(device) # device를 넣어야 하나?
            trg = batch.trg.to(device)

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

In [21]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
import time
import math
import random

# 일단 10epoch만 해봄(너무 오래걸린다.)
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 996m 28s
	Train Loss: 5.283 | Train PPL: 196.950
	Validation Loss: 5.949 | Validation PPL: 383.262
